En este archivo, voy a desarrollar la función que se me pide en el tercer endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_3.png "Title")

Importación de las librerías necesarias para trabajar

In [1]:
import pandas as pd

Lectura de los conjuntos de datos necesarios necesarios y breve vistazo de la cabecera de los mismos

In [2]:
user_items = pd.read_parquet("../EDA/user_items_complete.parquet")
user_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


In [3]:
steam_games = pd.read_parquet("../Datasets/steam_games_complete.parquet")
steam_games.head()

,item_id,item_name,developer,genres,tags,specs,release_date,price
88310,761140,Lost Summoner Kitty,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99
88311,643980,Ironbound,Secret Level SRL,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,0.00
88312,670290,Real Pool 3D - Poolians,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,0.00
88313,767400,弹炸人2222,彼岸领域,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player],2017-12-07,0.99
88315,772540,Battle Royale Trainer,Trickjump Games Ltd,"[Action, Adventure, Simulation]","[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",2018-01-04,3.99


Con estos Dataframe, analizo lo que me piden:
- A partir del id de un usuario, devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación, y la cantidad de items que tiene el usuario. Emtonces lo que necesito es:
    - id de un usuario
    - cantidad de dinero gastado por el usuario
    - porcentaje de recomendación
    - cantidad de items que tiene el usuario

Veamos las columnas que tenemos disponibles en todos nuestros conjuntos de datos.

In [5]:
print(user_reviews.columns)
print()
print(user_items.columns)
print()
print(steam_games.columns)
print()

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id'],
      dtype='object')

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id'],
      dtype='object')

Index(['id', 'app_name', 'developer', 'price', 'release_year', 'categorical'], dtype='object')



De acuerdo a estos resultados...
Las columnas que voy a usar para este trabajo son:
- user_reviews: user_id, recommend
- user_items: user_id, item_id
- steam_games: id = item_id, price.

El resto de columnas no me sirve, así que las descarto

In [6]:
for column in user_reviews.columns: # Por cada columna en el dataframe user_reviews
    if column not in ['user_id', 'recommend']: # Si la columna no es la que me sirve
        user_reviews.drop(column, axis=1, inplace=True) # La elimino
print(user_reviews.columns) # Muestro las columnas que me quedaron cómo forma de comprobar los resultados

# Repito lo mismo con los otros dos dataframe

for column in user_items.columns:
    if column not in ['user_id', 'item_id']:
        user_items.drop(column, axis=1, inplace=True)
print(user_items.columns)

for column in steam_games.columns:
    if column not in ['price', 'id']:
        steam_games.drop(column, axis=1, inplace=True)
print(steam_games.columns)


Index(['recommend', 'user_id'], dtype='object')
Index(['item_id', 'user_id'], dtype='object')
Index(['id', 'price'], dtype='object')


Hechemos un vistazo a cómo quedaron los dataframe

In [55]:
user_reviews

,recommend,user_id
0,True,76561197970982479
1,True,76561197970982479
2,True,76561197970982479
3,True,js41637
4,True,js41637
...,...,...
59300,True,76561198312638244
59301,True,76561198312638244
59302,True,LydiaMorley
59303,True,LydiaMorley


In [8]:
user_items

,item_id,user_id
0,10,76561197970982479
1,20,76561197970982479
2,30,76561197970982479
3,40,76561197970982479
4,50,76561197970982479
...,...,...
5153204,346330,76561198329548331
5153205,373330,76561198329548331
5153206,388490,76561198329548331
5153207,521570,76561198329548331


In [9]:
steam_games

,id,price
88310,761140,4.99
88311,643980,0.00
88312,670290,0.00
88313,767400,0.99
88315,772540,3.99
...,...,...
120439,745400,1.99
120440,773640,1.99
120441,733530,4.99
120442,610660,1.99


Primero que nada, hago un chequeo de nulos en ambos 3 dataframes

In [10]:
user_items.isnull().sum() # Ninguno, perfecto

item_id    0
user_id    0
dtype: int64

In [11]:
user_reviews.isnull().sum() # Ninguno, perfecto

recommend    0
user_id      0
dtype: int64

In [12]:
steam_games.isnull().sum() # Perfecto

id       0
price    0
dtype: int64

Ahora que ya tengo los dataframe con las columnas que necesito, lo que me resulta más fácil de determinar es cuantos juegos tiene cada usuario. Para ello, utilizo el dataframe user_items

In [13]:
user_items[user_items["user_id"] == "76561197970982479"]["user_id"].count()

277

Primer prototipo del endpoint

In [14]:
def user_data(user_id):
    cantidad_items = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    return cantidad_items

In [15]:
user_data(76561197970982479) # La salida es la esperada

277

En base a esta cantidad de juegos que tiene el usuario, puedo determinar el porcentaje de recomendación del mismo.

In [16]:
user_reviews[(user_reviews["user_id"] == "76561197970982479") & user_reviews["recommend"] == True]["recommend"].count()

3

Segundo prototipo del endpoint

In [17]:
def user_data(user_id):
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    return f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}% de Recomendación"

In [18]:
user_data(76561197970982479)

'1.08% de Recomendación'

Ahora necesito encontrar la cantidad de dinero gastado por el usuario, para ello, tengo que usar los dataframe user_items y steam_games, ya que en ellos se encuentran todos los datos que necesito

In [19]:
steam_games[steam_games["id"] == 10]

,id,price
120416,10,9.99


In [20]:
user_items[user_items["item_id"] == 10] # Tengo que cambiarle el tipo de dato a la columna item_id

,item_id,user_id


In [21]:
user_items["item_id"] = user_items["item_id"].astype(int)

Me doy cuenta que lo más sencillo sería trabajar en un mismo dataframe, por lo cuál decido unir tanto el de items cómo el de games

In [22]:
usuario_juego_precio = pd.merge(user_items, steam_games, how="left", left_on="item_id", right_on="id")
usuario_juego_precio

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
5153204,346330,76561198329548331,346330.0,0.00
5153205,373330,76561198329548331,NaN,NaN
5153206,388490,76561198329548331,388490.0,0.00
5153207,521570,76561198329548331,521570.0,0.00


Voy a tomarme la libertad de eliminar las filas que contienen nulos, ya que de las mismas, no puedo asegurar que el juego es gratuito, ni tampoco que sea pago, así que las elimino

In [23]:
usuario_juego_precio.isnull().sum()

item_id          0
user_id          0
id         1064014
price      1064014
dtype: int64

In [24]:
usuario_juego_precio.dropna(inplace=True)
usuario_juego_precio

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
5153202,304930,76561198329548331,304930.0,0.00
5153203,227940,76561198329548331,227940.0,0.00
5153204,346330,76561198329548331,346330.0,0.00
5153206,388490,76561198329548331,388490.0,0.00


Filtremos el dataset usando el mismo usuario con el cuál veniamos haciendo las pruebas

In [25]:
usuario_juego_precio[usuario_juego_precio["user_id"] == "76561197970982479"]

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
272,289130,76561197970982479,289130.0,29.99
273,268500,76561197970982479,268500.0,59.99
274,461640,76561197970982479,461640.0,1.99
275,379720,76561197970982479,379720.0,29.99


De aquí, la columna que más nos interesa es la del precio, del cual queremos conocer la suma, la cantidad total gastada por el usuario, así que...

In [26]:
usuario_juego_precio[usuario_juego_precio["user_id"] == "76561197970982479"]["price"].sum() # Ese usuario gastó más de 3400 dólares

3404.3300000000004

Tercer prototipo del endpoint

In [27]:
def user_data(user_id):
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()
    return f"El usuario gastó {round(cantidad_dinero_gastado, 2)} USD"

In [28]:
user_data(76561197970982479)

'El usuario gastó 3404.33 USD'

Cuarto prototipo del endpoint

In [29]:
def user_data(user_id):
    diccionario_de_retorno = {}
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

    diccionario_de_retorno["Usuario"] = user_id
    diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"
    diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
    diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
    
    return diccionario_de_retorno

In [30]:
user_data(76561197970982479)

{'Usuario': 76561197970982479,
 'Dinero gastado': '3404.33 USD',
 'Porcentaje de recomendación': '1.08%',
 'Cantidad de juegos': 277}

Bueno, se podría decir que ya resolví el endpoint 2, pero en realidad me faltan hacerle algunas aclaraciones y ajustes a la función para que su desempeño sea óptimo

In [32]:
user_data(76561197970982474) # Veo que cuando pongo un id que no corresponde a ningún usuario, me sale lo siguiente

C:\Users\crisr\AppData\Local\Temp\ipykernel_20384\1780711741.py:9: RuntimeWarning: invalid value encountered in scalar divide
  diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"


{'Usuario': 76561197970982474,
 'Dinero gastado': '0.0 USD',
 'Porcentaje de recomendación': 'nan%',
 'Cantidad de juegos': 0}

Quinto prototipo del endpoint

In [34]:
def user_data(user_id):
    diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

    if user_items[user_items["user_id"] == str(user_id)].empty:
        return "No se encontraron datos para el usuario con id: " + str(user_id)

    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    # cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    # cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

    # diccionario_de_retorno["Usuario"] = user_id
    # diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"
    # diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
    # diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
    
    return diccionario_de_retorno

In [35]:
user_data(76561197970982474) # La salida es la esperada.

'No se encontraron datos para el usuario con id: 76561197970982474'

Sexto prototipo de la función

In [36]:
def user_data(user_id):

    diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

    if user_items[user_items["user_id"] == str(user_id)].empty: # En caso de que no haya usuarios con el ID ingresado, se muestra lo siguiente:
        return "No se encontraron datos para el usuario con ID: " + str(user_id)
    else: # En caso de que haya usuarios con el ID ingresado, se muestra lo siguiente

        # Se cuenta la cantidad de juegos que tiene el usuario
        cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()

        # Se cuenta la cantidad de recomendaciones. En la columna recommend aparece "True" o "False", tuve en cuenta sólo las que aparece "True" del usuario ingresado.
        cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()

        # Se cuenta la cantidad de dinero gastado por el usuario, haciendo una suma de la columna "price" de los juegos que tiene el usuario. 
        cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

        # A partir del ID ingresado, del dinero gastado, del porcentaje de recomendación, y de la cantidad de juegos, se almacenan en el diccionario previamente creado con las claves correspondientes.
        diccionario_de_retorno["Usuario"] = user_id
        diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"

        # Si el usuario no hizo recomendaciones, para no tener errores de división por 0, defino que el porcentaje de recomendación es de 0%
        if cantidad_de_recomendaciones == 0:
            diccionario_de_retorno["Porcentaje de recomendaciones"] = "0%"
        else:
            # Si hizo recomendaciones, se hace el calculo normalmente
            diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
        
        diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
        
        # Se devuelve el diccionario creado
        return diccionario_de_retorno

In [37]:
user_data(76561197970982474) # La salida es la esperada.

'No se encontraron datos para el usuario con ID: 76561197970982474'

In [51]:
user_reviews[user_reviews["recommend"] == False] # Probemos con sandwiches1

,recommend,user_id
48,False,76561198043472122
62,False,76561198066046412
68,False,76561198070565427
84,False,boydeer
122,False,sandwiches1
...,...,...
59262,False,kimjongadam
59268,False,iwishihadaids
59283,False,MeloncraftLP
59284,False,76561198277602337


In [53]:
user_reviews[user_reviews["user_id"] == "sandwiches1"] # Ideal, sólo tiene una recomendación y es negativa, osea tiene 0 recomendaciones positivas

,recommend,user_id
122,False,sandwiches1


In [54]:
user_data("sandwiches1")

{'Usuario': 'sandwiches1',
 'Dinero gastado': '1528.9 USD',
 'Porcentaje de recomendaciones': '0%',
 'Cantidad de juegos': 172}

Exporto los dataframe en formato parquet con compresión snappy a la carpeta Datasets para importarlos luego desde allí

In [63]:
user_items.to_parquet("../Datasets/user_items_endpoint_2.parquet",compression="snappy")

In [64]:
user_reviews.to_parquet("../Datasets/user_reviews_endpoint_2.parquet", compression="snappy")

In [65]:
usuario_juego_precio.to_parquet("../Datasets/steam_games_and_users_endpoint_2.parquet", compression="snappy")

Septimo y último prototipo funcional y exportable, le agrego la importación de los archivos correspondientes.

In [ ]:
def user_data(user_id):
    # Cargamos los dataframes dentro de la función
    user_items = pd.read_parquet("./Datasets/user_items_endpoint_2.parquet")
    user_reviews = pd.read_parquet("./Datasets/user_reviews_endpoint_2.parquet")
    usuario_juego_precio = pd.read_parquet("./Datasets/steam_games_and_users_endpoint_2.parquet")

    diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

    if user_items[user_items["user_id"] == str(user_id)].empty: # En caso de que no haya usuarios con el ID ingresado, se muestra lo siguiente:
        return "No se encontraron datos para el usuario con ID: " + str(user_id)
    else: # En caso de que haya usuarios con el ID ingresado, se muestra lo siguiente

        # Se cuenta la cantidad de juegos que tiene el usuario
        cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()

        # Se cuenta la cantidad de recomendaciones. En la columna recommend aparece "True" o "False", tuve en cuenta sólo las que aparece "True" del usuario ingresado.
        cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()

        # Se cuenta la cantidad de dinero gastado por el usuario, haciendo una suma de la columna "price" de los juegos que tiene el usuario. 
        cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

        # A partir del ID ingresado, del dinero gastado, del porcentaje de recomendación, y de la cantidad de juegos, se almacenan en el diccionario previamente creado con las claves correspondientes.
        diccionario_de_retorno["Usuario"] = user_id
        diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"

        # Si el usuario no hizo recomendaciones, para no tener errores de división por 0, defino que el porcentaje de recomendación es de 0%
        if cantidad_de_recomendaciones == 0:
            diccionario_de_retorno["Porcentaje de recomendaciones"] = "0%"
        else:
            # Si hizo recomendaciones, se hace el calculo normalmente
            diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
        
        diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
        
        # Se devuelve el diccionario creado
        return diccionario_de_retorno